# What is the best day to buy stocks? 

This project seeks to prove whether the infamous "weekend effect" is true when it comes to the stock market. 

> The weekend effect is a term used by investors and brokers describing how stocks tend to trade at lower values and have better performances if bought on Mondays. 

Stocks observed: Vanguard 500 Index Fund (VOO)
- Dataset: https://finance.yahoo.com/quote/VOO/history?p=VOO
- Data range: Sep 09 2010 - Feb 22 2023



In [1]:
# import libraries

import numpy as np
import pandas as pd 
pd.options.mode.chained_assignment = None
import os 

In [2]:
# Read the data

df = pd.read_csv("VOO.csv", parse_dates=["Date"])

df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-09-09,102.500000,102.500000,101.139999,101.320000,80.010056,26500
1,2010-09-10,101.680000,101.860001,101.300003,101.779999,80.373260,8600
2,2010-09-13,102.959999,103.139999,102.500000,103.059998,81.384056,33750
3,2010-09-14,102.839996,103.480003,102.379997,103.040001,81.368286,59400
4,2010-09-15,102.620003,103.379997,102.400002,103.300003,81.573563,9250


In [3]:
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
3130,2023-02-15,377.149994,380.529999,376.329987,380.480011,380.480011,2831600
3131,2023-02-16,375.660004,379.429993,375.089996,375.190002,375.190002,3077500
3132,2023-02-17,373.179993,374.450012,371.320007,374.220001,374.220001,3282600
3133,2023-02-21,370.399994,371.339996,366.480011,366.790009,366.790009,3109200
3134,2023-02-22,367.179993,368.600006,364.839996,366.329987,366.329987,3872000


In [4]:
# Add a column for day of the week

day_mapper = {0: "Monday", 1: "Tuesday", 2: "Wednesday", 3: "Thursday", 4: "Friday", 5: "Saturday", 6: "Sunday"}

df["DayOfWeek"] = df["Date"].map(lambda x: day_mapper[x.dayofweek])

df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,DayOfWeek
0,2010-09-09,102.500000,102.500000,101.139999,101.320000,80.010056,26500,Thursday
1,2010-09-10,101.680000,101.860001,101.300003,101.779999,80.373260,8600,Friday
2,2010-09-13,102.959999,103.139999,102.500000,103.059998,81.384056,33750,Monday
3,2010-09-14,102.839996,103.480003,102.379997,103.040001,81.368286,59400,Tuesday
4,2010-09-15,102.620003,103.379997,102.400002,103.300003,81.573563,9250,Wednesday


# Method 1: Average dip per day

This method calculates the difference in Opening value between consecutive days and finds the average for each weekday. 

In [5]:
# Find difference between previous value and current value

df["diff_from_previous_day"] = (df["Open"].diff() / df["Open"]) * 100

df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,DayOfWeek,diff_from_previous_day
0,2010-09-09,102.500000,102.500000,101.139999,101.320000,80.010056,26500,Thursday,NaN
1,2010-09-10,101.680000,101.860001,101.300003,101.779999,80.373260,8600,Friday,-0.806452
2,2010-09-13,102.959999,103.139999,102.500000,103.059998,81.384056,33750,Monday,1.243200
3,2010-09-14,102.839996,103.480003,102.379997,103.040001,81.368286,59400,Tuesday,-0.116689
4,2010-09-15,102.620003,103.379997,102.400002,103.300003,81.573563,9250,Wednesday,-0.214376


In [6]:
# Compare days to find biggest dip

df.groupby("DayOfWeek")["diff_from_previous_day"].mean()

DayOfWeek
Friday       0.066238
Monday      -0.047267
Thursday    -0.012745
Tuesday      0.107526
Wednesday    0.054519
Name: diff_from_previous_day, dtype: float64

# Method 2: Backtester Method

For this method, a function is created which simulates a user trade given any amount for all 5 weekdays and returns the final amounts.  

In [7]:
# Create a table of every day between the minimum date and maximum date to deal including holidays

dates = pd.date_range(start=df["Date"].min(), end=df["Date"].max())

# Add the name of the day of the week

date_table = pd.DataFrame(data={"Calendar Date":dates})

date_table["Weekday"] = date_table["Calendar Date"].map(lambda x: day_mapper[x.dayofweek])

date_table

,Calendar Date,Weekday
0,2010-09-09,Thursday
1,2010-09-10,Friday
2,2010-09-11,Saturday
3,2010-09-12,Sunday
4,2010-09-13,Monday
...,...,...
4545,2023-02-18,Saturday
4546,2023-02-19,Sunday
4547,2023-02-20,Monday
4548,2023-02-21,Tuesday


In [8]:
# Create a table of all dates and remove the weekends (Joins)

full_calendar = pd.merge(left=date_table, right=df, how='left', left_on='Calendar Date', right_on='Date')

full_calendar = full_calendar[~full_calendar["Weekday"].isin(["Saturday", "Sunday"])]

full_calendar.rename(columns={"Date": "Trading Day"}, inplace=True)

full_calendar

,Calendar Date,Weekday,Trading Day,Open,High,Low,Close,Adj Close,Volume,DayOfWeek,diff_from_previous_day
0,2010-09-09,Thursday,2010-09-09,102.500000,102.500000,101.139999,101.320000,80.010056,26500.0,Thursday,NaN
1,2010-09-10,Friday,2010-09-10,101.680000,101.860001,101.300003,101.779999,80.373260,8600.0,Friday,-0.806452
4,2010-09-13,Monday,2010-09-13,102.959999,103.139999,102.500000,103.059998,81.384056,33750.0,Monday,1.243200
5,2010-09-14,Tuesday,2010-09-14,102.839996,103.480003,102.379997,103.040001,81.368286,59400.0,Tuesday,-0.116689
6,2010-09-15,Wednesday,2010-09-15,102.620003,103.379997,102.400002,103.300003,81.573563,9250.0,Wednesday,-0.214376
...,...,...,...,...,...,...,...,...,...,...,...
4543,2023-02-16,Thursday,2023-02-16,375.660004,379.429993,375.089996,375.190002,375.190002,3077500.0,Thursday,-0.396633
4544,2023-02-17,Friday,2023-02-17,373.179993,374.450012,371.320007,374.220001,374.220001,3282600.0,Friday,-0.664562
4547,2023-02-20,Monday,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4548,2023-02-21,Tuesday,2023-02-21,370.399994,371.339996,366.480011,366.790009,366.790009,3109200.0,Tuesday,-0.750540


In [9]:
# Take the next day's opening value as buy price for empty rows

full_calendar = full_calendar.bfill(axis='rows').reset_index(drop=True)

# Check total number of trades for each weekday

full_calendar["Weekday"].value_counts()

Thursday     650
Friday       650
Monday       650
Tuesday      650
Wednesday    650
Name: Weekday, dtype: int64

In [10]:
# Create a backtest function to simulate trading for each weekday

def day_backtester(day, investment_amount, data):
    temp_data = data[data["Weekday"] == day]
    temp_data["Shares Owned"] = investment_amount / temp_data["Open"]
    final_price = temp_data["Open"].iloc[-1]
    final_amount = temp_data["Shares Owned"].sum() * final_price
    final_amount_formatted = "${:,.2f}".format(final_amount)

    return round(final_amount_formatted,2)

In [153]:
# Create for loop to loop through each weekday

for i in full_calendar["Weekday"].unique():
    print(f"{i}:", day_backtester(i, 800, full_calendar))

Thursday: $996,549.19
Friday: $989,360.84
Monday: $982,039.43
Tuesday: $981,396.88
Wednesday: $972,055.89


# Conclusion

During this project the weekend effect was both proved and disproved. 

- During Method 1, we found that the average dip value was the lowest on Monday sitting at an average dip of -0.047267. This proved that the weekend effect does hold some truth as Monday was the best day to invest due to having the lowest opening price. 

- However, during Method 2, while using the `day_backtester` function, the results indicated that trades placed on Thursdays appeared to have the highest returns over time. This indicated that stocks bought-in on Thursday's tended to perform much better than on any other day of the week.   